In [41]:
import json
from pulp import *
from datetime import datetime, timedelta
from pprint import pprint

In [13]:
import json

def load_json_data(file_path):
  """
  Loads data from a JSON file.

  Args:
    file_path: Path to the JSON file.

  Returns:
    A Python dictionary or list representing the JSON data.
  """
  try:
    with open(file_path, 'r') as f:
      data = json.load(f)
    return data
  except FileNotFoundError:
    print(f"Error: File not found: {file_path}")
    return None
  except json.JSONDecodeError as e:
    print(f"Error: Invalid JSON in file: {file_path}")
    print(f"Error message: {e}")
    return None

In [27]:
def generate_time_slots(start_time="08:00", end_time="20:00", interval=60):
    """
    Generates a list of time slots between start_time and end_time.

    Args:
        start_time (str): Start time in "HH:MM" format.
        end_time (str): End time in "HH:MM" format.
        interval (int): Interval in minutes between time slots.

    Returns:
        List[str]: List of time slots as "HH:MM".
    """
    slots = []
    start = datetime.strptime(start_time, "%H:%M")
    end = datetime.strptime(end_time, "%H:%M")
    current = start
    while current < end:
        slots.append(current.strftime("%H:%M"))
        current += timedelta(minutes=interval)
    return slots

In [31]:
def get_instructor_by_name(instructors, name):
    # Iterate through the list of instructors and return the one with the matching name
    for instructor in instructors:
        if instructor["name"] == name:
            return instructor
    return None  # If no instructor is found with that name

In [34]:
instructors = [
        {"name": "AEL", "lessons": ["Math 101", "Physics 201"], "preferred_days": ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]},
        {"name": "Jane Smith", "lessons": ["Biology 101", "Chemistry 201"], "preferred_days": ["Tuesday", "Thursday", "Friday"]},
        {"name": "Alice Johnson", "lessons": ["English 101", "History 201"], "preferred_days": ["Monday", "Tuesday", "Thursday"]},
        {"name": "Robert Brown", "lessons": ["Philosophy 101", "Math 102"], "preferred_days": ["Monday", "Wednesday", "Friday"]},
        {"name": "Laura Wilson", "lessons": ["Physics 301", "Chemistry 301"], "preferred_days": ["Tuesday"]}  # Not available any day
    ]

In [36]:
get_instructor_by_name(instructors,"AEL")['preferred_days']

['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']

In [ ]:
from ortools.linear_solver import pywraplp

def solve_schedule():
    # Solver'ı oluşturuyoruz
    solver = pywraplp.Solver.CreateSolver('SCIP')
    if not solver:
        return

    # Veriler (girdi verisi)
    instructors = [
        {"name": "AEL", "lessons": ["Math101"], "preferred_days": ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]},
        {"name": "BANUDIRI", "lessons": ["Math101,Chemistry301"], "preferred_days": ["Tuesday", "Wednesday", "Friday"]},
        {"name": "JaneSmith", "lessons": ["Biology101", "Philosophy101"], "preferred_days": ["Tuesday", "Thursday", "Friday"]},
        {"name": "JohnDoe", "lessons": ["Physics201", "Philosophy101"], "preferred_days": ["Tuesday", "Thursday", "Friday"]},
        {"name": "AliceJohnson", "lessons": ["English101", "History201"], "preferred_days": ["Monday", "Tuesday", "Thursday"]},
        {"name": "RobertBrown", "lessons": ["Philosophy101", "Math102"], "preferred_days": ["Monday", "Wednesday", "Friday"]},
        {"name": "LauraWilson", "lessons": ["Chemistry301"], "preferred_days": ["Tuesday"]}  # Only teaching Chemistry301
    ]


    lessons = [
        {"name": "Math101", "instructors": ["AEL", "BANUDIRI"], "grade": 1, "type": "FaceToFace", "duration": 3},
        {"name": "Biology101", "instructors": ["JaneSmith"], "grade": 2, "type": "Online", "duration": 3},
        {"name": "English101", "instructors": ["AliceJohnson"], "grade": 4, "type": "FaceToFace", "duration": 3},
        {"name": "Physics201", "instructors": ["JohnDoe"], "grade": 2, "type": "FaceToFace", "duration": 3},
        {"name": "Philosophy101", "instructors": ["RobertBrown", "JohnDoe", "JaneSmith"], "grade": 1, "type": "FaceToFace", "duration": 3},
        {"name": "History201", "instructors": ["AliceJohnson"], "grade": 2, "type": "Hybrid", "duration": 3},
        {"name": "Math102", "instructors": ["RobertBrown"], "grade": 1, "type": "FaceToFace", "duration": 3},
        {"name": "Chemistry301", "instructors": ["LauraWilson","BANUDIRI"], "grade": 3, "type": "Online", "duration": 3}
    ]


    classrooms = ["Room101", "Lab202", "Room103", "Lab204", "Room205", "Lab206", "Room207", "Lab208"]
    time_slots = [f"{hour}:00" for hour in range(8, 20)]  # 8:00 AM to 8:00 PM
    days_of_week = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
    grades = [1, 2, 3, 4]

    # Değişkenler: x[ders, ogretmen, sinif, gun, zaman]
    x = {}

    # Tüm dersler, öğretmenler, sınıflar, günler ve zaman dilimleri için anahtarlar oluşturuluyor
    for lesson in lessons:
        for instructor in lesson['instructors']:
            for classroom in classrooms:
                for day in get_instructor_by_name(instructors,instructor)['preferred_days']:
                    for time in time_slots:
                        print(f'x_{lesson["name"]}_{lesson['grade']}_{instructor}_{classroom}_{day}_{time}')
                        x[(lesson['name'],lesson['grade'], instructor, classroom, day, time)] = solver.BoolVar(f'x_{lesson["name"]}_{lesson['grade']}_{instructor}_{classroom}_{day}_{time}')

    print("Atamalar tamamlandi --------------------------------------")

    # # Kısıtlar:
    # # 1. Aynı sınıfta aynı zaman diliminde yalnızca bir ders olabilir
    for grade in grades:
        for day in days_of_week:
            for time in time_slots:
                total = 0 
                for lesson in lessons:
                    if lesson['grade'] == grade:
                        for instructor in lesson['instructors']:
                            for classroom in classrooms:
                                if (lesson['name'], lesson['grade'], instructor, classroom, day, time) in x:
                                    print("add constraint 1")
                                    total += x[(lesson['name'],lesson['grade'], instructor, classroom, day, time)]
                solver.Add(total <= 1)



    # 2. Her dersin bir öğretmeni olmalı
    for lesson in lessons:
        total = 0
        for instructor in lesson['instructors']:
            for classroom in classrooms:
                for day in days_of_week:
                    for time in time_slots:
                        if (lesson['name'],lesson['grade'], instructor, classroom, day, time) in x:
                            print("add constraint 2")
                            total += x[(lesson['name'],lesson["grade"], instructor, classroom, day, time)]
        solver.Add(total == 1)

    print("2. kisit tamamlandi --------------------------------------")
                

    3. Öğretmen yalnızca tercih ettiği günlerde atanabilir
    for instructor in instructors:
        for lesson in instructor['lessons']:
            for day in days_of_week:
                if day not in instructor['preferred_days']:
                    for time in time_slots:
                        solver.Add(
                            sum(x[(lesson, instructor['name'], classroom, day, time)] for classroom in classrooms) == 0
                        )
    print("3. kisit tamamlandi --------------------------------------")



    # 4. Online derslerin sınıfı olamaz ancak mutlaka öğretmeni olmalı
    for lesson in lessons:
        if lesson['type'] in ["FaceToFace" , "Hybrid"]:
            # hybrid and face to face lessons have to have a classroom
            for day in days_of_week:
                for time in time_slots:
                    total = 0
                    for instructor in lesson['instructors']:
                        for classroom in classrooms:
                            if (lesson['name'],lesson['grade'], instructor, classroom, day, time) in x:
                                total += x[(lesson['name'],lesson['grade'], instructor, classroom, day, time)]
                                print("add constraint 4 face to face, hybrid")
                    solver.Add(total == 1)
        else:
            for day in days_of_week:
                for time in time_slots:
                    total = 0
                    for instructor in lesson['instructors']:
                        for classroom in classrooms:
                            if (lesson['name'], lesson['grade'], instructor, classroom, day, time) in x:
                                total += x[(lesson['name'], lesson['grade'], instructor, classroom, day, time)]
                                print("add constraint 4 online")
                    solver.Add(total == 0)  # No classroom for online lessons

    print("4. kisit tamamlandi --------------------------------------")



    # 5. Hybrid ve yüz yüze derslere mutlaka öğretmen ve sınıf atanmalı
    for lesson in lessons:
        if lesson['type'] in ["Hybrid", "Face To Face"]:
            # Hybrid and Face-to-Face lessons must have both a teacher and a classroom assigned
            for day in days_of_week:
                for time in time_slots:
                    total = 0
                    for instructor in lesson['instructors']:
                        for classroom in classrooms:
                            # Check if the lesson, instructor, classroom, day, and time slot combination exists in x
                            if (lesson['name'], lesson['grade'], instructor, classroom, day, time) in x:
                                total += x[(lesson['name'], lesson['grade'], instructor, classroom, day, time)]
                                print("add constraint 5")
                    # Add the constraint that the total sum must be 1, ensuring both a teacher and classroom are assigned
                    solver.Add(total == 1)

    print("5. kisit tamamlandi --------------------------------------")



    # 6. Her dersin ders saati boyunca ilgili sınıf ve öğretmen boş olmalı
    for lesson in lessons:
        for instructor in lesson['instructors']:
            for classroom in classrooms:
                for day in days_of_week:
                    for start_time in range(8, 20 - lesson['duration'] + 1):  # Consider all possible start times for the lesson
                        total = 0
                        # Ensure the classroom and instructor are free for the entire duration of the lesson
                        for time_slot in range(start_time, start_time + lesson['duration']):
                            if (lesson['name'], lesson['grade'], instructor, classroom, day, time_slot) in x:
                                total += x[(lesson['name'], lesson['grade'], instructor, classroom, day, time_slot)]
                                print("add constraint 6")
                        # The sum of decision variables for the time slots of this lesson must equal 1
                        # This ensures that the classroom and instructor are occupied only for the lesson's duration
                        solver.Add(total == 1)


    print("6. kisit tamamlandi --------------------------------------")

    # Amaç Fonksiyonu: Burada örnekte amaç fonksiyonu olmadığından 0 olarak bırakıyoruz
    # solver.Minimize(0)

    # Çözüm bulma
    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        for lesson in lessons:
            for instructor in lesson['instructors']:
                for classroom in classrooms:
                    for day in days_of_week:
                        for time in time_slots:
                            if x[(lesson['name'],lesson['grade'], instructor, classroom, day, time)].solution_value() == 1:
                                print(f'{lesson["name"]} - {instructor} - {classroom} - {day} - {time}:00')
    else:
        print('Çözüm bulunamadı.')


solve_schedule()


x_Math101_1_AEL_Room101_Monday_8:00
x_Math101_1_AEL_Room101_Monday_9:00
x_Math101_1_AEL_Room101_Monday_10:00
x_Math101_1_AEL_Room101_Monday_11:00
x_Math101_1_AEL_Room101_Monday_12:00
x_Math101_1_AEL_Room101_Monday_13:00
x_Math101_1_AEL_Room101_Monday_14:00
x_Math101_1_AEL_Room101_Monday_15:00
x_Math101_1_AEL_Room101_Monday_16:00
x_Math101_1_AEL_Room101_Monday_17:00
x_Math101_1_AEL_Room101_Monday_18:00
x_Math101_1_AEL_Room101_Monday_19:00
x_Math101_1_AEL_Room101_Tuesday_8:00
x_Math101_1_AEL_Room101_Tuesday_9:00
x_Math101_1_AEL_Room101_Tuesday_10:00
x_Math101_1_AEL_Room101_Tuesday_11:00
x_Math101_1_AEL_Room101_Tuesday_12:00
x_Math101_1_AEL_Room101_Tuesday_13:00
x_Math101_1_AEL_Room101_Tuesday_14:00
x_Math101_1_AEL_Room101_Tuesday_15:00
x_Math101_1_AEL_Room101_Tuesday_16:00
x_Math101_1_AEL_Room101_Tuesday_17:00
x_Math101_1_AEL_Room101_Tuesday_18:00
x_Math101_1_AEL_Room101_Tuesday_19:00
x_Math101_1_AEL_Room101_Wednesday_8:00
x_Math101_1_AEL_Room101_Wednesday_9:00
x_Math101_1_AEL_Room101_We

In [132]:
from ortools.linear_solver import pywraplp

def get_instructor_by_name(instructors, name):
    for instructor in instructors:
        if instructor["name"] == name:
            return instructor
    return None

def solve_schedule():
    solver = pywraplp.Solver.CreateSolver('SCIP')
    if not solver:
        return

    # Input data
    instructors = [
        {"name": "AEL", "lessons": ["Math101"], "preferred_days": ["Tuesday", "Wednesday", "Thursday", "Friday"]},
        {"name": "BANUDIRI", "lessons": ["Math101", "Chemistry301"], "preferred_days": ["Tuesday", "Wednesday", "Friday"]},
        {"name": "JaneSmith", "lessons": ["Biology101", "Philosophy101"], "preferred_days": ["Tuesday", "Thursday", "Friday"]},
        {"name": "JohnDoe", "lessons": ["Physics201", "Philosophy101"], "preferred_days": ["Tuesday", "Thursday", "Friday"]},
        {"name": "AliceJohnson", "lessons": ["English101", "History201"], "preferred_days": ["Monday", "Tuesday", "Thursday"]},
        {"name": "RobertBrown", "lessons": ["Philosophy101", "Math102"], "preferred_days": ["Monday", "Wednesday", "Friday"]},
        {"name": "LauraWilson", "lessons": ["Chemistry301"], "preferred_days": ["Tuesday"]}
    ]

    lessons = [
        {"name": "Math101", "instructors": ["AEL", "BANUDIRI"], "grade": 1, "type": "FaceToFace", "duration": 3},
        {"name": "Biology101", "instructors": ["JaneSmith"], "grade": 2, "type": "Online", "duration": 3},
        {"name": "English101", "instructors": ["AliceJohnson"], "grade": 4, "type": "FaceToFace", "duration": 2},
        {"name": "Physics201", "instructors": ["JohnDoe"], "grade": 2, "type": "FaceToFace", "duration": 3},
        {"name": "Philosophy101", "instructors": ["RobertBrown", "JohnDoe", "JaneSmith"], "grade": 1, "type": "FaceToFace", "duration": 1},
        {"name": "History201", "instructors": ["AliceJohnson"], "grade": 2, "type": "Hybrid", "duration": 2},
        {"name": "Math102", "instructors": ["RobertBrown"], "grade": 1, "type": "FaceToFace", "duration": 3},
        {"name": "Chemistry301", "instructors": ["LauraWilson", "BANUDIRI"], "grade": 3, "type": "Online", "duration": 3}
    ]

    classrooms = ["Room101", "Lab202", "Room103", "Lab204", "Room205", "Lab206", "Room207", "Lab208"]
    time_slots = list(range(8, 17))  # 8:00 AM to 4:00 PM
    days_of_week = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
    grades = [1, 2, 3, 4]

    # Decision variables - only for start times
    x = {}
    for lesson in lessons:
        for instructor in lesson['instructors']:
            instructor_days = get_instructor_by_name(instructors, instructor)['preferred_days']
            for day in instructor_days:
                # Only create variables for valid start times
                for time in range(8, 17 - lesson['duration']):
                    if lesson['type'] in ["FaceToFace", "Hybrid"]:
                        for classroom in classrooms:
                            x[(lesson['name'], instructor, classroom, day, time)] = solver.BoolVar(
                                f'x_{lesson["name"]}_{instructor}_{classroom}_{day}_{time}'
                            )
                    else:  # Online classes don't need classrooms
                        x[(lesson['name'], instructor, None, day, time)] = solver.BoolVar(
                            f'x_{lesson["name"]}_{instructor}_None_{day}_{time}'
                        )

    # Constraint 1: Each lesson must be scheduled exactly once
    for lesson in lessons:
        constraint_expr = []
        for instructor in lesson['instructors']:
            instructor_days = get_instructor_by_name(instructors, instructor)['preferred_days']
            for day in instructor_days:
                for time in range(8, 17 - lesson['duration']):
                    if lesson['type'] in ["FaceToFace", "Hybrid"]:
                        for classroom in classrooms:
                            if (lesson['name'], instructor, classroom, day, time) in x:
                                constraint_expr.append(x[(lesson['name'], instructor, classroom, day, time)])
                    else:
                        if (lesson['name'], instructor, None, day, time) in x:
                            constraint_expr.append(x[(lesson['name'], instructor, None, day, time)])
        solver.Add(sum(constraint_expr) == 1)

    # Constraint 2: No instructor can teach during overlapping time slots
    for instructor in instructors:
        for day in days_of_week:
            for time in time_slots:
                constraint_expr = []
                for lesson in lessons:
                    if instructor['name'] in lesson['instructors']:
                        # Check all possible start times that would overlap with current time slot
                        for start_time in range(max(8, time - lesson['duration'] + 1), min(time + 1, 17 - lesson['duration'])):
                            if lesson['type'] in ["FaceToFace", "Hybrid"]:
                                for classroom in classrooms:
                                    if (lesson['name'], instructor['name'], classroom, day, start_time) in x:
                                        constraint_expr.append(x[(lesson['name'], instructor['name'], classroom, day, start_time)])
                            else:
                                if (lesson['name'], instructor['name'], None, day, start_time) in x:
                                    constraint_expr.append(x[(lesson['name'], instructor['name'], None, day, start_time)])
                if constraint_expr:
                    solver.Add(sum(constraint_expr) <= 1)

    # Constraint 3: No classroom can have overlapping classes
    for classroom in classrooms:
        for day in days_of_week:
            for time in time_slots:
                constraint_expr = []
                for lesson in lessons:
                    if lesson['type'] in ["FaceToFace", "Hybrid"]:
                        # Check all possible start times that would overlap with current time slot
                        for start_time in range(max(8, time - lesson['duration'] + 1), min(time + 1, 17 - lesson['duration'])):
                            for instructor in lesson['instructors']:
                                if (lesson['name'], instructor, classroom, day, start_time) in x:
                                    constraint_expr.append(x[(lesson['name'], instructor, classroom, day, start_time)])
                if constraint_expr:
                    solver.Add(sum(constraint_expr) <= 1)

    # Constraint 4: No overlapping classes for the same grade level
    for grade in grades:
        for day in days_of_week:
            for time in time_slots:
                constraint_expr = []
                grade_lessons = [lesson for lesson in lessons if lesson['grade'] == grade]
                for lesson in grade_lessons:
                    # Check all possible start times that would overlap with current time slot
                    for start_time in range(max(8, time - lesson['duration'] + 1), min(time + 1, 17 - lesson['duration'])):
                        for instructor in lesson['instructors']:
                            if lesson['type'] in ["FaceToFace", "Hybrid"]:
                                for classroom in classrooms:
                                    if (lesson['name'], instructor, classroom, day, start_time) in x:
                                        constraint_expr.append(x[(lesson['name'], instructor, classroom, day, start_time)])
                            else:
                                if (lesson['name'], instructor, None, day, start_time) in x:
                                    constraint_expr.append(x[(lesson['name'], instructor, None, day, start_time)])
                if constraint_expr:
                    solver.Add(sum(constraint_expr) <= 1)

    # Solve
    status = solver.Solve()

    # Print results
    if status == pywraplp.Solver.OPTIMAL:
        print("\nOptimal schedule found:\n")
        for lesson in lessons:
            for instructor in lesson['instructors']:
                for day in days_of_week:
                    for time in range(8, 17 - lesson['duration']):
                        if lesson['type'] in ["FaceToFace", "Hybrid"]:
                            for classroom in classrooms:
                                if (lesson['name'], instructor, classroom, day, time) in x:
                                    if x[(lesson['name'], instructor, classroom, day, time)].solution_value() == 1:
                                        print(f"Lesson: {lesson['name']} (Grade {lesson['grade']}) - Instructor: {instructor}")
                                        print(f"Day: {day}, Time: {time}:00 - {time + lesson['duration']}:00")
                                        print(f"Type: {lesson['type']}, Classroom: {classroom}\n")
                        else:
                            if (lesson['name'], instructor, None, day, time) in x:
                                if x[(lesson['name'], instructor, None, day, time)].solution_value() == 1:
                                    print(f"Lesson: {lesson['name']} (Grade {lesson['grade']}) - Instructor: {instructor}")
                                    print(f"Day: {day}, Time: {time}:00 - {time + lesson['duration']}:00")
                                    print(f"Type: {lesson['type']} (No classroom needed)\n")
    else:
        print("No optimal solution found.")

if __name__ == "__main__":
    solve_schedule()


Optimal schedule found:

Lesson: Math101 (Grade 1) - Instructor: AEL
Day: Tuesday, Time: 8:00 - 11:00
Type: FaceToFace, Classroom: Room101

Lesson: Biology101 (Grade 2) - Instructor: JaneSmith
Day: Tuesday, Time: 11:00 - 14:00
Type: Online (No classroom needed)

Lesson: English101 (Grade 4) - Instructor: AliceJohnson
Day: Monday, Time: 8:00 - 10:00
Type: FaceToFace, Classroom: Lab202

Lesson: Physics201 (Grade 2) - Instructor: JohnDoe
Day: Tuesday, Time: 8:00 - 11:00
Type: FaceToFace, Classroom: Lab202

Lesson: Philosophy101 (Grade 1) - Instructor: RobertBrown
Day: Monday, Time: 8:00 - 9:00
Type: FaceToFace, Classroom: Room101

Lesson: History201 (Grade 2) - Instructor: AliceJohnson
Day: Monday, Time: 10:00 - 12:00
Type: Hybrid, Classroom: Room101

Lesson: Math102 (Grade 1) - Instructor: RobertBrown
Day: Monday, Time: 9:00 - 12:00
Type: FaceToFace, Classroom: Room103

Lesson: Chemistry301 (Grade 3) - Instructor: LauraWilson
Day: Tuesday, Time: 8:00 - 11:00
Type: Online (No classroom n

In [136]:
output = [
    {
        "name": "Math101",
        "grade": 1,
        "instructor": "AEL",
        "day": "Tuesday",
        "start_time": 8,
        "duration": 3,
        "type": "FaceToFace",
        "classroom": "Room101"
    },
    {
        "name": "Biology101",
        "grade": 2,
        "instructor": "JaneSmith",
        "day": "Tuesday",
        "start_time": 11,
        "duration": 3,
        "type": "Online",
        "classroom": "None"
    },
    {
        "name": "English101",
        "grade": 4,
        "instructor": "AliceJohnson",
        "day": "Monday",
        "start_time": 8,
        "duration": 2,
        "type": "FaceToFace",
        "classroom": "Lab202"
    },
    {
        "name": "Physics201",
        "grade": 2,
        "instructor": "JohnDoe",
        "day": "Tuesday",
        "start_time": 8,
        "duration": 3,
        "type": "FaceToFace",
        "classroom": "Lab202"
    },
    {
        "name": "Philosophy101",
        "grade": 1,
        "instructor": "RobertBrown",
        "day": "Monday",
        "start_time": 8,
        "duration": 1,
        "type": "FaceToFace",
        "classroom": "Room101"
    },
    {
        "name": "History201",
        "grade": 2,
        "instructor": "AliceJohnson",
        "day": "Monday",
        "start_time": 10,
        "duration": 2,
        "type": "Hybrid",
        "classroom": "Room101"
    },
    {
        "name": "Math102",
        "grade": 1,
        "instructor": "RobertBrown",
        "day": "Monday",
        "start_time": 9,
        "duration": 3,
        "type": "FaceToFace",
        "classroom": "Room103"
    },
    {
        "name": "Chemistry301",
        "grade": 3,
        "instructor": "LauraWilson",
        "day": "Tuesday",
        "start_time": 8,
        "duration": 3,
        "type": "Online",
        "classroom": "None"
    }
]


In [153]:
def generate_schedule_pdf(solution_data, output_path="schedule.pdf"):
    """
    Generates a PDF schedule with grades as columns and days/times as rows.
    """
    css_content = """
        @page {
            size: A3 portrait;
            margin: 1cm;
        }
        
        body {
            font-family: Arial, sans-serif;
        }
        
        table {
            width: 100%;
            border-collapse: collapse;
            font-size: 8pt;
        }
        
        th, td {
            border: 1px solid black;
            padding: 1px;
            text-align: center;
            vertical-align: top;
            height: 12px;
        }
        
        td:empty {
            height: 8px;
            padding: 0;
        }
        
        .time-cell {
            width: 40px;
            background-color: #f0f0f0;
            font-size: 7pt;
            height: 12px;
            padding: 1px;
        }
        
        .grade-header {
            background-color: #e0e0e0;
            font-weight: bold;
            font-size: 9pt;
            height: 15px;
        }
        
        .course-cell {
            min-height: 15px;
            padding: 2px;
        }
        
        .face-to-face {
            background-color: #90EE90;
        }
        
        .online {
            background-color: #ADD8E6;
        }
        
        .hybrid {
            background-color: #FFB6C1;
        }
        
        .course-code {
            font-weight: bold;
            color: red;
            margin-bottom: 1px;
            font-size: 8pt;
        }
        
        .course-instructor {
            font-style: italic;
            font-size: 7pt;
        }
        
        .course-details {
            font-size: 7pt;
        }
        
        .title {
            text-align: center;
            font-size: 11pt;
            font-weight: bold;
            margin: 8px 0;
            background-color: yellow;
            padding: 3px;
        }
    """

    # Constants
    days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
    grades = [1, 2, 3, 4]
    times = list(range(8, 21))  # 8:00 to 20:00

    # Create schedule grid
    schedule_grid = {day: {grade: {time: None for time in times} for grade in grades} for day in days}

    # Populate schedule grid
    for lesson in solution_data:
        day = lesson["day"]
        grade = lesson["grade"]
        start_time = lesson["start_time"]
        for t in range(start_time, start_time + lesson["duration"]):
            schedule_grid[day][grade][t] = lesson

    # Generate HTML
    html_content = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <meta charset="UTF-8">
    </head>
    <body>
        <div class="title">YTÜ Bilgisayar Mühendisliği Bölümü Ders Programı</div>
        <table>
            <tr>
                <th class="time-cell"></th>
    """

    # Add grade headers
    for grade in grades:
        html_content += f'<th class="grade-header">{grade}. Grade</th>'
    html_content += "</tr>"

    # Generate rows for each day and time
    for day in days:
        first_row_of_day = True
        for time in times:
            row_html = "<tr>"

            # Add time cell with day name if it's first row of the day
            if first_row_of_day:
                row_html += f'<td class="time-cell">{day}<br/>{time:02d}:00</td>'
                first_row_of_day = False
            else:
                row_html += f'<td class="time-cell">{time:02d}:00</td>'

            # Add cells for each grade
            for grade in grades:
                lesson = schedule_grid[day][grade][time]

                if lesson is None:
                    row_html += "<td></td>"
                elif time == lesson["start_time"]:
                    # Only create cell content at the start time
                    course_type_class = f"course-cell {lesson['type'].lower().replace(' ', '-')}"
                    row_html += f"""
                        <td rowspan="{lesson['duration']}" class="{course_type_class}">
                            <div class="course-code">{lesson['name']} ({lesson['duration']} hour)</div>
                            <div class="course-instructor">{lesson['instructor']}</div>
                            <div class="course-details">
                                ({lesson['type']})
                                {f"<br>{lesson['classroom']}" if lesson['classroom'] else ""}
                            </div>
                        </td>
                    """
                # Skip cell if it's covered by a rowspan from above

            row_html += "</tr>"
            html_content += row_html

    html_content += """
        </table>
    </body>
    </html>
    """

    # Create temporary files for HTML and CSS
    with tempfile.NamedTemporaryFile(suffix=".html", mode="w", encoding="utf-8", delete=False) as html_file:
        html_file.write(html_content)
        html_path = html_file.name

    with tempfile.NamedTemporaryFile(suffix=".css", mode="w", encoding="utf-8", delete=False) as css_file:
        css_file.write(css_content)
        css_path = css_file.name

    try:
        # Generate PDF
        HTML(html_path).write_pdf(output_path, stylesheets=[CSS(filename=css_path)])
    finally:
        # Clean up temporary files
        os.unlink(html_path)
        os.unlink(css_path)

In [154]:
# Example usage:
# formatted_solution = format_solver_solution(output)
generate_schedule_pdf(output, 'course_schedule.pdf')